In [34]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, simps
from mpl_toolkits.mplot3d import axes3d
# import pymsteams
from time import time, localtime, strftime
from datetime import datetime
import os
# from scipy.interpolate import RegularGridInterpolator as rgi, interp1d
plt.rcParams['figure.figsize'] = [8, 4]
# plt.rcParams.update({'font.size' : 18})

# path = r'/home/teamgem/Heriot-Watt University Team Dropbox/RES_EPS_Quantum_Photonics_Lab/Experiments/Current Experiments/Visible Quantum Memory/GEM and Stark shift/20240819_GEM_MBE_sims/'
path = r'C:\Users\fgbse\Heriot-Watt University Team Dropbox\RES_EPS_Quantum_Photonics_Lab\Experiments\Current Experiments\Visible Quantum Memory\GEM and Stark shift\20240902_GEM_MBE_sims'
# path = r'C:\Users\Finley Giles-Book\Heriot-Watt University Team Dropbox\RES_EPS_Quantum_Photonics_Lab\Experiments\Current Experiments\Visible Quantum Memory\GEM and Stark shift\20240815_GEM_MBE_sims'

In [35]:
def lorentz(x,delta,lw,grad,stark,length):
    line_shape = 1/((np.pi * (lw + grad*length/2*stark)*(1 + ((delta + grad*stark*x)/(lw + grad*length/2*stark))**2)))
    return line_shape

def rect_abs_prof(x,delta,lw,OD,grad,stark,length):
    if np.size(x) == 1:
        line_shape =  np.where(abs(delta - grad*stark*x) <= (lw), OD/length, 0)
        return line_shape
    else: 
        line_shape = np.zeros_like(delta)
        for xi in x:
            single_line_shape = np.where(abs(delta - grad*stark*xi) <= (lw), OD/length, 0)
            line_shape += single_line_shape
        return line_shape

def gauss_abs_prof(x,delta,lw,OD,grad,stark,length):
    if np.size(x) == 1:
        line_shape = 1/((lw + grad*length/2*stark)*np.sqrt(2*np.pi)) * np.exp(-1/2 * (delta + grad*stark*x)**2/(lw)**2)
        return line_shape
    else: 
        line_shape = np.zeros_like(delta)
        for xi in x:
            single_line_shape = 1/((lw + grad*length/2*stark)*np.sqrt(2*np.pi)) * np.exp(-1/2 * (delta + grad*stark*xi)**2/(lw)**2)
            line_shape += single_line_shape
        return line_shape
    
def lorentz_abs_prof(x,delta,lw,OD,grad,stark,length):
    if np.size(x) == 1:
        line_shape = 1/((np.pi * (lw + grad*length/2*stark)*(1 + ((delta + grad*stark*x)/(lw + grad*length/2*stark))**2)))
        return line_shape
    else: 
        line_shape = np.zeros_like(delta)
        for xi in x:
            single_line_shape = 1/((np.pi * (lw + grad*length/2*stark)*(1 + ((delta + grad*stark*xi)/(lw + grad*length/2*stark))**2)))
            line_shape += single_line_shape
        return line_shape

def grad_switch(t,t_switch):
    if t > t_switch:
        return 1
    else:
        return -1
    
def gauss(t,t_peak,amp,sigma):    
    line_shape = amp*np.exp(-(t - t_peak)**2 / (2 * sigma**2))
    return line_shape

def interpolate_E_t(E,t_arr,tn):

    interp = np.interp(tn,t_arr,E)
    return interp

def MBE_pol_z(t, alpha, E, t_arr, delta, x_n, grad, stark, t_switch, g, gamma, w):
    
    Ez = interpolate_E_t(E,t_arr,t)

    dalpha_dt = -(gamma/2)*alpha - (1j) * (delta + grad * stark * grad_switch(t_switch,t) * x_n)*alpha - (1j) * g * w * Ez

    return dalpha_dt

In [38]:
def MBE_field(x, E, delta, t0, t1, alpha0, E0, t_arr, grad, stark, t_switch, g, gamma, w0, OD, lw, L):

    # Calculate the optical field over dz
    # Since the the function calculates the spatial derivative of 
    # E and the eqn does not include any terms with E then then initial value of E is not needed
    # rhs_int is the initial value

    # E1 = E0-E[:,0]*np.log(-OD)
    # E1 = E0-E[:,0]
    # E1 = E0
    # E1 = E[:,0]
    E1 = E0*np.exp(-OD*x/L)

    # print(np.shape(E[:,0]))

    sol_alpha_z = solve_ivp(MBE_pol_z,
                            (t0, t1),
                            alpha0,
                            args=(E1, t_arr, delta, x, grad, stark, t_switch, g, gamma, w0),
                            t_eval=t_arr,
                            dense_output=True,
                            vectorized=False,
                            method='RK45')
        
    alpha = sol_alpha_z.y.T

    dE_dx = -simps(gauss_abs_prof(x,delta,lw,OD,grad,stark,L)*alpha,axis=1)

    print(x)

    return dE_dx

In [39]:
# Parameters - converted assuming that k0/2e0 = lambda (wavelength) = 1

gamma = 0.31#0.01#0.31                            # Polarization decay rate
g = 406#2000#406                                 # Transition coupling strength
t_input = 2                                 # Gaussian pulse peak time
sigma = 0.5/np.sqrt(2*np.log(2))        # Gaussian pulse sigma (time domain)
OD = 1                              # Optical depth used in Lorentzian
grad = 1E6#7.37e-11                         # Electric field gradient
stark = 6.74e-10                        # Stark coefficient
lw = 0.1                                  # Zero field absorption linewidth
t_switch = 5                            # Gradient direction switch time
A = 3.5e-3#-3                              # Electric field magnitude
L = 16475/2                               # Optical path length
t_total = 10
w0 = -1                                 # Atomic excitation field

# Discretisation

Nx = 100          # Spatial grid  
x_arr = np.linspace(-L/2, L/2, Nx)
x0 = x_arr[0]
x1 = x_arr[-1]

Nt = 100            # Time grid
t_arr = np.linspace(0,t_total, Nt)
t0 = t_arr[0]
t1 = t_arr[-1]

Nd = 100
delta_range = 20  # Detuning grid
delta = np.linspace(-delta_range/2, delta_range/2, Nd)
d0 = delta[0]
d1 = delta[-1]

# Initialisation

alpha0 = np.zeros(Nd, dtype=np.complex128)
E0 = np.complex128(gauss(t_arr,t_input,A,sigma))

alpha = np.zeros((Nd, Nt, Nx), dtype=np.complex128)
E = np.zeros((Nt, Nx), dtype=np.complex128)
P = np.zeros((Nt, Nx), dtype=np.complex128)

sol = solve_ivp(MBE_field,
                (x0, x1),
                E0,
                args=(delta, t0, t1, alpha0, E0, t_arr, grad, stark, t_switch, g, gamma, w0, OD, lw, L),
                t_eval=x_arr,
                dense_output=True,
                vectorized=True,
                method='RK45')
    
E = sol.y[0,:]

-4118.75
-4118.749987776253
-4118.749755525052
-4118.749633287578
-4118.749022100207
-4118.748913444675
-4118.748777625259
-4118.748777625259
-4118.746332875778
-4118.745110501037
-4118.738998627333
-4118.737912072008
-4118.736553877851
-4118.736553877851
-4118.7121063830355
-4118.699882635628
-4118.638763898588
-4118.627898345337
-4118.614316403772
-4118.614316403772
-4118.369841455614
-4118.247603981535
-4117.636416611139
-4117.527761078624
-4117.391941662981
-4117.391941662981
-4116.871815785048
-4116.611752846081
-4115.311438151249
-4115.08027109439
-4114.791312273316
-4114.791312273316
-4116.975579062454
-4116.767397762191
-4115.726491260874
-4115.541441216196
-4115.310128660348
-4115.310128660348
-4114.893766059821
-4114.685584759558
-4113.644678258242
-4113.4596282135635
-4113.228315657715
-4113.228315657715
-4112.7651741100235
-4112.533603336178
-4111.375749466949
-4111.169908779086
-4110.912607919257
-4110.912607919257
-4110.48256736023
-4110.267547080716
-4109.1924456831475
-